# Import Libraries

In [1]:
import streamlit as st
from pathlib import Path
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error

#test by alex

In [2]:
# Import Dataset
df = pd.read_excel('data_excel.xlsx')

In [3]:
# check 5 data teratas
df.head()

,kd_lpse,nama_lpse,kd_tender,nama_paket,pagu,hps,ang,kualifikasi_paket,kategori_pengadaan,mtd_pemilihan,mtd_evaluasi,mtd_kualifikasi,kontrak_pembayaran,nilai_penawaran,nilai_terkoreksi,nilai_negosiasi,nilai_reverse_auction,nilai_kontrak
0,10,LPSE Kota Surabaya,10682010,"Biaya Manajemen Konstruksi (Tidak Sederhana), ...",1147520000,1.139435e+09,APBD,Menengah,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,1.037300e+09,1.037300e+09,1.034385e+09,0.0,1.034385e+09
1,10,LPSE Kota Surabaya,10683010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",297000000,2.967800e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,2.475550e+08,2.475550e+08,2.472250e+08,0.0,2.472250e+08
2,10,LPSE Kota Surabaya,10684010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",297000000,2.967800e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,2.750110e+08,2.750110e+08,2.748570e+08,0.0,2.748570e+08
3,10,LPSE Kota Surabaya,10685010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",176000000,1.710500e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,1.235300e+08,1.235300e+08,1.222100e+08,0.0,1.222100e+08
4,10,LPSE Kota Surabaya,10690010,Pekerjaan Pengangkutan Sampah Surabaya Timur II,5561005049,4.697102e+09,APBD,Perusahaan Non Kecil,Jasa Lainnya,Tender,Sistem Gugur,Pasca Satu File,Harga Satuan,4.692706e+09,4.692706e+09,4.689403e+09,0.0,4.689403e+09


In [4]:
df.isnull().sum()

kd_lpse                     0
nama_lpse                   0
kd_tender                   0
nama_paket                  0
pagu                        0
hps                         0
ang                         0
kualifikasi_paket        3705
kategori_pengadaan          0
mtd_pemilihan               0
mtd_evaluasi                0
mtd_kualifikasi             0
kontrak_pembayaran          0
nilai_penawaran          1029
nilai_terkoreksi         1029
nilai_negosiasi          1029
nilai_reverse_auction    1029
nilai_kontrak            1029
dtype: int64

In [5]:
df['kualifikasi_paket']
# df['nilai_terkoreksi']
# df['nilai_penawaran']
# df['nilai_negosiasi']
# df['nilai_reverse_auction']
# df['nilai_kontrak']

0                    Menengah
1            Perusahaan Kecil
2            Perusahaan Kecil
3            Perusahaan Kecil
4        Perusahaan Non Kecil
                 ...         
91454        Perusahaan Kecil
91455        Perusahaan Kecil
91456        Perusahaan Kecil
91457        Perusahaan Kecil
91458        Perusahaan Kecil
Name: kualifikasi_paket, Length: 91459, dtype: object

In [34]:
# Mengambil kolom yang diinginkan dari DataFrame
kolom_yang_diinginkan = ["pagu", "hps", "kategori_pengadaan", "nilai_penawaran", "nilai_terkoreksi", "nilai_negosiasi", "nilai_kontrak"]
df = df[kolom_yang_diinginkan]

In [35]:
df

,pagu,hps,kategori_pengadaan,nilai_penawaran,nilai_terkoreksi,nilai_negosiasi,nilai_kontrak
0,1147520000,1.139435e+09,Jasa Konsultansi Badan Usaha Konstruksi,1.037300e+09,1.037300e+09,1.034385e+09,1.034385e+09
1,297000000,2.967800e+08,Jasa Konsultansi Badan Usaha Konstruksi,2.475550e+08,2.475550e+08,2.472250e+08,2.472250e+08
2,297000000,2.967800e+08,Jasa Konsultansi Badan Usaha Konstruksi,2.750110e+08,2.750110e+08,2.748570e+08,2.748570e+08
3,176000000,1.710500e+08,Jasa Konsultansi Badan Usaha Konstruksi,1.235300e+08,1.235300e+08,1.222100e+08,1.222100e+08
4,5561005049,4.697102e+09,Jasa Lainnya,4.692706e+09,4.692706e+09,4.689403e+09,4.689403e+09
...,...,...,...,...,...,...,...
91454,1150000000,1.149991e+09,Pekerjaan Konstruksi,1.108964e+09,1.108964e+09,0.000000e+00,1.108964e+09
91455,380000000,3.799916e+08,Pekerjaan Konstruksi,3.325000e+08,3.325000e+08,3.291700e+08,3.291700e+08
91456,700000000,6.999880e+08,Pekerjaan Konstruksi,5.220834e+08,5.220834e+08,0.000000e+00,5.220834e+08
91457,650000000,6.499871e+08,Pekerjaan Konstruksi,5.199897e+08,5.199897e+08,0.000000e+00,5.199897e+08


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91459 entries, 0 to 91458
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pagu                91459 non-null  int64  
 1   hps                 91459 non-null  float64
 2   kategori_pengadaan  91459 non-null  object 
 3   nilai_penawaran     90430 non-null  float64
 4   nilai_terkoreksi    90430 non-null  float64
 5   nilai_negosiasi     90430 non-null  float64
 6   nilai_kontrak       90430 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.9+ MB


In [6]:
df.isnull().sum()

kd_lpse                     0
nama_lpse                   0
kd_tender                   0
nama_paket                  0
pagu                        0
hps                         0
ang                         0
kualifikasi_paket        3705
kategori_pengadaan          0
mtd_pemilihan               0
mtd_evaluasi                0
mtd_kualifikasi             0
kontrak_pembayaran          0
nilai_penawaran          1029
nilai_terkoreksi         1029
nilai_negosiasi          1029
nilai_reverse_auction    1029
nilai_kontrak            1029
dtype: int64

In [7]:
#deletion on rows with missing value
print("number of rows before deletion:", len(df))
print("number of rows after deletion:", len(df.dropna()))
dataclean = df.dropna()

number of rows before deletion: 91459
number of rows after deletion: 86818


In [8]:
# Calculate the percentage of null values for each column
percentage_null = (dataclean.isnull().sum() / len(dataclean)) * 100

# Create a DataFrame to display the percentage of null values and sort it
null_summary = pd.DataFrame({
    'Column': percentage_null.index,
    'Percentage_Null': percentage_null.values
})


# Sort the DataFrame by the Percentage_Null column in descending order
null_summary = null_summary.sort_values(by='Percentage_Null', ascending=False)

# Display the sorted DataFrame
print(null_summary)

                   Column  Percentage_Null
0                 kd_lpse              0.0
1               nama_lpse              0.0
16  nilai_reverse_auction              0.0
15        nilai_negosiasi              0.0
14       nilai_terkoreksi              0.0
13        nilai_penawaran              0.0
12     kontrak_pembayaran              0.0
11        mtd_kualifikasi              0.0
10           mtd_evaluasi              0.0
9           mtd_pemilihan              0.0
8      kategori_pengadaan              0.0
7       kualifikasi_paket              0.0
6                     ang              0.0
5                     hps              0.0
4                    pagu              0.0
3              nama_paket              0.0
2               kd_tender              0.0
17          nilai_kontrak              0.0


In [9]:
dataclean = dataclean.drop_duplicates()

In [10]:
dataclean = dataclean.dropna()

In [11]:
dataclean

,kd_lpse,nama_lpse,kd_tender,nama_paket,pagu,hps,ang,kualifikasi_paket,kategori_pengadaan,mtd_pemilihan,mtd_evaluasi,mtd_kualifikasi,kontrak_pembayaran,nilai_penawaran,nilai_terkoreksi,nilai_negosiasi,nilai_reverse_auction,nilai_kontrak
0,10,LPSE Kota Surabaya,10682010,"Biaya Manajemen Konstruksi (Tidak Sederhana), ...",1147520000,1.139435e+09,APBD,Menengah,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,1.037300e+09,1.037300e+09,1.034385e+09,0.0,1.034385e+09
1,10,LPSE Kota Surabaya,10683010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",297000000,2.967800e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,2.475550e+08,2.475550e+08,2.472250e+08,0.0,2.472250e+08
2,10,LPSE Kota Surabaya,10684010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",297000000,2.967800e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,2.750110e+08,2.750110e+08,2.748570e+08,0.0,2.748570e+08
3,10,LPSE Kota Surabaya,10685010,"Biaya Pengawasan Fisik (Tidak Sederhana), Nila...",176000000,1.710500e+08,APBD,Perusahaan Kecil,Jasa Konsultansi Badan Usaha Konstruksi,Seleksi,Kualitas dan Biaya,Pra Dua File Kualitas Biaya,Waktu Penugasan,1.235300e+08,1.235300e+08,1.222100e+08,0.0,1.222100e+08
4,10,LPSE Kota Surabaya,10690010,Pekerjaan Pengangkutan Sampah Surabaya Timur II,5561005049,4.697102e+09,APBD,Perusahaan Non Kecil,Jasa Lainnya,Tender,Sistem Gugur,Pasca Satu File,Harga Satuan,4.692706e+09,4.692706e+09,4.689403e+09,0.0,4.689403e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91454,802,SPSE Nasional,87875802,Pemeliharaan Berkala / Peningkatan Jalan Kampu...,1150000000,1.149991e+09,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,1.108964e+09,1.108964e+09,0.000000e+00,0.0,1.108964e+09
91455,802,SPSE Nasional,878786802,Rehabilitasi Box Culvert Way Kali Toman Kampun...,380000000,3.799916e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,3.325000e+08,3.325000e+08,3.291700e+08,0.0,3.291700e+08
91456,802,SPSE Nasional,878787802,Pemeliharaan Berkala/Rehabilitasi Ruas Jalan P...,700000000,6.999880e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,5.220834e+08,5.220834e+08,0.000000e+00,0.0,5.220834e+08
91457,802,SPSE Nasional,878788802,Pemeliharaan Berkala/Peningkatan Jalan Penghub...,650000000,6.499871e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,5.199897e+08,5.199897e+08,0.000000e+00,0.0,5.199897e+08


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [12]:
#check 5 data terbawah
df.tail()

,kd_lpse,nama_lpse,kd_tender,nama_paket,pagu,hps,ang,kualifikasi_paket,kategori_pengadaan,mtd_pemilihan,mtd_evaluasi,mtd_kualifikasi,kontrak_pembayaran,nilai_penawaran,nilai_terkoreksi,nilai_negosiasi,nilai_reverse_auction,nilai_kontrak
91454,802,SPSE Nasional,87875802,Pemeliharaan Berkala / Peningkatan Jalan Kampu...,1150000000,1.149991e+09,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,1.108964e+09,1.108964e+09,0.0,0.0,1.108964e+09
91455,802,SPSE Nasional,878786802,Rehabilitasi Box Culvert Way Kali Toman Kampun...,380000000,3.799916e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,3.325000e+08,3.325000e+08,329170000.0,0.0,3.291700e+08
91456,802,SPSE Nasional,878787802,Pemeliharaan Berkala/Rehabilitasi Ruas Jalan P...,700000000,6.999880e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,5.220834e+08,5.220834e+08,0.0,0.0,5.220834e+08
91457,802,SPSE Nasional,878788802,Pemeliharaan Berkala/Peningkatan Jalan Penghub...,650000000,6.499871e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,5.199897e+08,5.199897e+08,0.0,0.0,5.199897e+08
91458,802,SPSE Nasional,878789802,Pembangunan Pagar Keliling dan Drainase Di Sek...,420000000,4.199984e+08,APBD,Perusahaan Kecil,Pekerjaan Konstruksi,Tender,Sistem Gugur,Pasca Satu File,Gabungan Lumsum dan Harga Satuan,3.359987e+08,3.359987e+08,0.0,0.0,3.359987e+08


In [ ]:
#check tipe data
df.info()

In [ ]:
# Mengambil kolom yang diinginkan dari DataFrame
kolom_yang_diinginkan = ["id", "score", "ad_id", "count", "vasTags", "title", "localityId", "raw", "main_info"]
df = df[kolom_yang_diinginkan]


In [ ]:
df.isnull().sum()


In [ ]:
df['localityId']

In [ ]:
df['vasTags']

In [ ]:
# Calculate the percentage of null values for each column
# kegunaan menghitung missing value untuk mengetahui banyaknya data hilang, jika hanya < bisa dilakukan delete data. Diatas 1% harus menggunakan metode statistik (mean, median, modus) | backfill, frontfill, interpolasi
percentage_null = (df.isnull().sum() / len(df)) * 100

# Create a DataFrame to display the percentage of null values and sort it
null_summary = pd.DataFrame({
    'Column': percentage_null.index,
    'Percentage_Null': percentage_null.values
})



In [ ]:

# Sort the DataFrame by the Percentage_Null column in descending order
null_summary = null_summary.sort_values(by='Percentage_Null', ascending=False)

# Display the sorted DataFrame
print(null_summary)


In [ ]:


# Menghitung modus dari masing-masing kolom
modus_vasTags = df['vasTags'].mode()[0]
modus_localityId = df['localityId'].mode()[0]

# Mengganti nilai-nilai yang hilang dengan modus
df['vasTags'].fillna(modus_vasTags, inplace=True)
df['localityId'].fillna(modus_localityId, inplace=True)


## Check Missing Value

In [ ]:
# Calculate the percentage of null values for each column
percentage_null = (df.isnull().sum() / len(df)) * 100

# Create a DataFrame to display the percentage of null values and sort it
null_summary = pd.DataFrame({
    'Column': percentage_null.index,
    'Percentage_Null': percentage_null.values
})


# Sort the DataFrame by the Percentage_Null column in descending order
null_summary = null_summary.sort_values(by='Percentage_Null', ascending=False)

# Display the sorted DataFrame
print(null_summary)


# Data Cleaning

Untuk saat ini dilakukan drop duplikat dan missing values.

Nantinya data cleaning dilakukan sesuai dengan konteks datanya.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
import pandas as pd

# Misalkan df adalah DataFrame Anda
# df = ...

# Mengganti nama kolom
df = df.rename(columns={
    'id': 'ID',
    'score': 'SKOR',
    'ad_id': 'ID_DETAIL',
    'count': 'JUMLAH',
    'vasTags': 'FEATURE_TAMBAHAN',
    'title': 'JENIS_BARANG',
    'localityId': 'ID_DETAIL_BARANG',
    'raw': 'HARGA',
    'main_info': 'TAHUN_PRODUK_RELEASE'
})

# Tampilkan DataFrame setelah penggantian nama kolom
print(df)


In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Misalkan df adalah DataFrame Anda
# df = ...
# karna stdev mempengaruhi model

# Lakukan Label Encoding untuk kolom-kolom kategorikal
#
label_encoder = LabelEncoder()

for column in [ 'ID','SKOR','ID_DETAIL','JUMLAH','FEATURE_TAMBAHAN','JENIS_BARANG','ID_DETAIL_BARANG','HARGA','TAHUN_PRODUK_RELEASE']:
    df[column] = label_encoder.fit_transform(df[column])


In [ ]:
data_awal=df

In [ ]:
data_awal

In [ ]:
# Lakukan join berdasarkan indeks
result = data_awal.merge(df, left_index=True, right_index=True, how='left')

# Tampilkan hasil join
print(result)

In [ ]:
result.to_excel('result.xlsx')

# Linear Regression

Sebelum melakukan prediksi, terlebih dahulu harus ditentukan mana kolom yang menjadi feature dan target.

Dalam hal ini kita akan menggunakan `qty_pembalian` sebagai target dan lainnya sebagai feature.

## Split Feature dan Target

In [ ]:
# Hapus kolom 'qty_pembelian' dan 'Date' dari DataFrame
X = df.drop(columns=['HARGA'])


In [ ]:
y = df[['HARGA']] # Target yang akan diprediksi

In [ ]:
X.head()

In [ ]:
y.head()

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_train.shape,  X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_test

## Model Training

In [ ]:
from sklearn.linear_model import LinearRegression

# Tentukan model yang akan digunakan
lin_reg = LinearRegression()

# Train Model
lin_reg.fit(X_train, y_train)

In [ ]:
# retrieve the coefficients
data = X_train
model = lin_reg

coef_df = pd.DataFrame({
    'feature':['intercept'] + data.columns.tolist(),
    'coefficient':[model.intercept_[0]] + list(model.coef_[0])
})

coef_df


# Multiple Regression

## Split Feature dan Target

In [ ]:
# Hapus kolom 'qty_pembelian' dan 'Date' dari DataFrame
X = df.drop(columns=['HARGA'])


In [ ]:
y = df[['HARGA']] # Target yang akan diprediksi

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_train.shape,  X_test.shape, y_train.shape, y_test.shape

## Multicollinearity Check

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# heatmap correlation
data_train = pd.concat([X_train, y_train], axis=1)
corr = data_train.corr()

plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True, fmt='.2f')
plt.show()

In [ ]:
# calculate VIF scores for each feature
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant

X_vif = add_constant(X_train)

vif_df = pd.DataFrame([vif(X_vif.values, i)
               for i in range(X_vif.shape[1])],
              index=X_vif.columns).reset_index()
vif_df.columns = ['feature','vif_score']
vif_df = vif_df.loc[vif_df.feature!='const']
vif_df

In [ ]:
X_train

In [ ]:
X_train = X_train.drop(columns=['ID'])


In [ ]:

X_test = X_test.drop(columns=['ID'])

In [ ]:


# heatmap correlation
data_train = pd.concat([X_train, y_train], axis=1)
corr = data_train.corr()

plt.figure(figsize=(10,7))
sns.heatmap(corr, annot=True, fmt='.2f')
plt.show()

In [ ]:
# calculate VIF scores for each feature

X_vif = add_constant(X_train)

vif_df = pd.DataFrame([vif(X_vif.values, i)
               for i in range(X_vif.shape[1])],
              index=X_vif.columns).reset_index()
vif_df.columns = ['feature','vif_score']
vif_df = vif_df.loc[vif_df.feature!='const']
vif_df

Tidak ada vif score yang tinggi

## Model Training

In [ ]:
from sklearn.linear_model import LinearRegression

# Tentukan model yang akan digunakan
multi_reg = LinearRegression()

# Train Model
multi_reg.fit(X_train, y_train)

## Model Evaluation

### Training Error

Prediksi data train menggunakan model yang sudah dilatih

In [ ]:
y_predict_train = multi_reg.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print('RMSE for training data is {}'.format(np.sqrt(mean_squared_error(y_predict_train, y_train))))

### Testing Error

Prediksi data test menggunakan model yang sudah dilatih

In [ ]:
y_predict_test = multi_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

print('RMSE for testing data is {}'.format(np.sqrt(mean_squared_error(y_predict_test, y_test))))

In [ ]:
import numpy as np

# Anggaplah X_test dan y_predict_test adalah array NumPy
X_test_with_predictions = np.column_stack((X_test, y_predict_test))


In [ ]:
import pandas as pd

# Anggaplah X_test dan y_predict_test adalah array NumPy
# atau dataframe pandas, tergantung pada format data yang Anda miliki
X_test_with_predictions = pd.DataFrame(data=np.column_stack((X_test, y_predict_test)),
                                       columns=list(X_test.columns) + ['PREDIKSI'])


In [ ]:
X_test_with_predictions 

In [ ]:

# Membulatkan nilai kolom "Predicted" ke bilangan bulat
X_test_with_predictions['PREDIKSI'] = X_test_with_predictions['PREDIKSI'].round().astype(int)

# Tampilkan DataFrame setelah membulatkan nilai
print(X_test_with_predictions)


In [ ]:
y_test

In [ ]:

# Anggaplah X_test dan y_predict_test adalah array NumPy
# atau dataframe pandas, tergantung pada format data yang Anda miliki
X_akhir = pd.DataFrame(data=np.column_stack((X_test_with_predictions, y_test)),
                                       columns=list(X_test_with_predictions.columns) + ['AKTUAL'])


In [ ]:
X_akhir

In [ ]:
import matplotlib.pyplot as plt

# Dataframe

# Scatter plot
plt.scatter(X_akhir['PREDIKSI'], X_akhir['AKTUAL'])
plt.title('Scatter Plot Predicted vs Aktual')
plt.xlabel('Predicted')
plt.ylabel('Aktual')
plt.show()


In [ ]:
import matplotlib.pyplot as plt


# Line chart
plt.plot(X_akhir['PREDIKSI'], label='PREDIKSI', marker='o')
plt.plot(X_akhir['AKTUAL'], label='AKTUAL', marker='o')
plt.title('Line Chart Predicted vs Aktual')
plt.xlabel('Observation')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
X_akhir

In [ ]:
result

In [ ]:
# Ambil nilai unik dari kolom Harga_x dan Harga_y
unik_harga_x = result['HARGA_x'].unique()
unik_harga_y = result['HARGA_y'].unique()

# Tampilkan hasil
print("Nilai unik Harga_x:", unik_harga_x)
print("Nilai unik Harga_y:", unik_harga_y)

In [ ]:
# Buat DataFrame baru untuk menyimpan hasil unik
df_unik = pd.DataFrame({
    'Harga_x_unik': unik_harga_x,
    'Harga_y_unik': unik_harga_y
})

# Tampilkan hasil
print(df_unik)

In [ ]:
# Urutkan DataFrame berdasarkan Harga_x_unik
df_unik_sorted = df_unik.sort_values(by='Harga_x_unik')

# Tampilkan hasil
print(df_unik_sorted)

In [ ]:
X_akhir

In [ ]:
X_train

Jika Anda ingin mengubah data yang diberikan menjadi format yang dapat digunakan untuk prediksi, langkah-langkahnya akan melibatkan beberapa tahapan seperti encoding data kategorikal dan menyesuaikan struktur data. Berikut adalah contoh langkah-langkahnya:

1. Encoding Data Kategorikal:
        Lakukan encoding pada kolom-kolom yang bersifat kategorikal seperti 'JENIS_BARANG' dan 'FEATURE_TAMBAHAN'.

2. Menghilangkan Kolom yang Tidak Diperlukan:
        Hilangkan kolom-kolom yang tidak diperlukan untuk prediksi.

3. Membuat Struktur DataFrame yang Sesuai:
        Struktur DataFrame yang dihasilkan harus sesuai dengan struktur data yang digunakan saat melatih model.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Data baru yang belum diencode

data_baru = pd.DataFrame({
    'SKOR': [1.000, 1.000, 1.000],
    'ID_DETAIL': [905030584, 905030584, 905030584],
    'JUMLAH': [66, 61, 66],
    'FEATURE_TAMBAHAN': ['warranty', 'moneyback', 'moneyback'],
    'JENIS_BARANG': ['Mitsubishi Pajero Sport 2.4 Dakar 4x2 Solar-AT...', 'Mitsubishi Pajero Sport 2.4 Dakar 4x2 Solar-AT...', 'Mitsubishi Pajero Sport 2.4 Dakar 4x2 Solar-AT...'],
    'ID_DETAIL_BARANG': [5001329.0, 5001329.0, 5001329.0],
    'TAHUN_PRODUK_RELEASE': [2016, 2016, 2023]
})
# Lakukan Label Encoding untuk kolom-kolom kategorikal
label_encoder = LabelEncoder()

for column in [ 'SKOR','ID_DETAIL','JUMLAH','FEATURE_TAMBAHAN','JENIS_BARANG','ID_DETAIL_BARANG','TAHUN_PRODUK_RELEASE']:
    data_baru[column] = label_encoder.fit_transform(data_baru[column])

# Gunakan model untuk memprediksi harga
y_pred = multi_reg.predict(data_baru)


In [ ]:
data_baru

In [ ]:
y_pred